In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime

In [54]:
#CREATE FUNCTIONS TO CONVERT STRING DATA TO NUMERIC TO LATER USE FOR MODELING AND ANALYSIS

def get_numeric_value(value_str):
    try:
        return float(value_str)
    except ValueError:
        return None

def convert_height(height_str):
    height_str = height_str.replace('"', '')
    feet, inches = map(int, height_str.split("'"))
    total_inches = feet * 12 + inches
    return total_inches

def convert_weight(weight_str):
    return float(weight_str.replace('lbs.', '').strip())

def convert_reach(reach_str):
    if reach_str == '--':
        return None
    return float(reach_str.replace('"', '').strip())

def convert_percentage(percentage_str):
    try:
        numeric_value = float(percentage_str.replace('%', '')) / 100
        return numeric_value
    except ValueError:
        return None

def fighter_stats(fighter_url):
    fighter_page = requests.get(fighter_url)
    fighter_soup = BeautifulSoup(fighter_page.content, 'html.parser')

    fighter_data = {}
    fighter_physical_stats = fighter_soup.find('div', class_='b-list__info-box')
    if fighter_physical_stats:
        physical_details = fighter_physical_stats.find_all('li', class_='b-list__box-list-item')
        for detail in physical_details:
            label = detail.find('i', class_='b-list__box-item-title').get_text(strip=True)
            value = detail.get_text(strip=True).replace(label, '').strip()

            if label == 'Height:':
                fighter_data[label] = convert_height(value)
            elif label == 'Weight:':
                fighter_data[label] = convert_weight(value)
            elif label == 'Reach:':
                fighter_data[label] = convert_reach(value)
            else:
                fighter_data[label] = value

    fighter_career_stats = fighter_soup.find('div', class_='b-list__info-box-left')
    if fighter_career_stats:
        career_stats = fighter_career_stats.find_all('li', class_='b-list__box-list-item')
        for stat in career_stats:
            label_element = stat.find('i', class_='b-list__box-item-title')
            value = stat.get_text(strip=True).replace(label_element.get_text(strip=True), '').strip()
            label = label_element.get_text(strip=True).rstrip(':')
            fighter_data[label] = value

            if label in ['Str. Acc.', 'Str. Def', 'TD Acc.', 'TD Def.']:
                new_label = f"{label} (%)"
                converted_value = convert_percentage(value)
                fighter_data[new_label] = converted_value
                del fighter_data[label]
            else:
                fighter_data[label] = value

    fighter_df = pd.DataFrame([fighter_data])

    cols_to_strip_percent = ['Str. Acc. (%)', 'Str. Def (%)', 'TD Acc. (%)', 'TD Def. (%)']
    fighter_df[cols_to_strip_percent] = fighter_df[cols_to_strip_percent].replace('%', '', regex=True)

    return fighter_df


In [55]:
fighter_url = 'http://www.ufcstats.com/fighter-details/029eaff01e6bb8f0'

In [56]:
import datetime

def fighter_stats(fighter_url):
    fighter_page = requests.get(fighter_url)
    fighter_soup = BeautifulSoup(fighter_page.content, 'html.parser')

    fighter_data = {}
    fighter_physical_stats = fighter_soup.find('div', class_='b-list__info-box')
    if fighter_physical_stats:
        physical_details = fighter_physical_stats.find_all('li', class_='b-list__box-list-item')
        for detail in physical_details:
            label = detail.find('i', class_='b-list__box-item-title').get_text(strip=True)
            value = detail.get_text(strip=True).replace(label, '').strip()

            if label == 'Height:':
                fighter_data[label] = convert_height(value)
            elif label == 'Weight:':
                fighter_data[label] = convert_weight(value)
            elif label == 'Reach:':
                fighter_data[label] = convert_reach(value)
            else:
                fighter_data[label] = value

    fighter_career_stats = fighter_soup.find('div', class_='b-list__info-box-left')
    if fighter_career_stats:
        career_stats = fighter_career_stats.find_all('li', class_='b-list__box-list-item')
        for stat in career_stats:
            label_element = stat.find('i', class_='b-list__box-item-title')
            value = stat.get_text(strip=True).replace(label_element.get_text(strip=True), '').strip()
            label = label_element.get_text(strip=True).rstrip(':')

            if label in ['Str. Acc.', 'Str. Def', 'TD Acc.', 'TD Def.']:
                new_label = f"{label} (%)"
                converted_value = convert_percentage(value)
                fighter_data[new_label] = converted_value
            else:
                fighter_data[label] = value

    cols_to_strip_percent = ['Str. Acc. (%)', 'Str. Def (%)', 'TD Acc. (%)', 'TD Def. (%)']
    fighter_df = pd.DataFrame([fighter_data])
    fighter_df[cols_to_strip_percent] = fighter_df[cols_to_strip_percent].replace('%', '', regex=True)
    
    return fighter_df

In [62]:
import datetime

def get_opponent_stats(fighter_url):
    opponent_urls = []

    fighter_page = requests.get(fighter_url)
    fighter_soup = BeautifulSoup(fighter_page.content, 'html.parser')

    recent_fights_table = fighter_soup.find('table')
    recent_fights_rows = recent_fights_table.find_all('tr')

    for row in recent_fights_rows:
        fighter_column = row.find('td', class_='b-fight-details__table-col l-page_align_left')
        
        if fighter_column:
            fighter_links = fighter_column.find_all('a', class_='b-link b-link_style_black')
            if len(fighter_links) >= 2:
                opponent_url = fighter_links[1]['href']
                opponent_urls.append(opponent_url)
    
    opponent_stats_dfs = []
    for opponent_url in opponent_urls:
        opponent_stats_df = fighter_stats(opponent_url)
        opponent_stats_dfs.append(opponent_stats_df)
    
    combined_opponent_stats = pd.concat(opponent_stats_dfs, ignore_index=True)

    combined_opponent_stats['DOB:'] = pd.to_datetime(combined_opponent_stats['DOB:'], format='%b %d, %Y')
    today = datetime.datetime.today()
    combined_opponent_stats['Age'] = (today - combined_opponent_stats['DOB:']).apply(lambda x: x.days // 365)
    combined_opponent_stats.drop(columns=['DOB:'], inplace=True)
    return combined_opponent_stats

x = get_opponent_stats(fighter_url)


In [63]:
get_opponent_stats('http://www.ufcstats.com/fighter-details/0052de90691d4a93')

,Height:,Weight:,Reach:,STANCE:,SLpM,Str. Acc. (%),SApM,Str. Def (%),,TD Avg.,TD Acc. (%),TD Def. (%),Sub. Avg.,Age
0,67,155.0,72.0,Orthodox,3.82,0.48,1.91,0.54,,3.43,0.36,0.75,0.0,26
1,70,155.0,70.0,Southpaw,2.35,0.59,1.27,0.61,,3.24,0.62,0.90,1.1,31
2,70,155.0,71.0,Orthodox,3.83,0.46,3.37,0.56,,1.55,0.52,0.63,0.0,31
3,69,155.0,72.0,Orthodox,1.65,0.37,3.72,0.46,,7.08,0.42,0.00,0.0,37
4,66,155.0,66.0,Southpaw,2.65,0.35,3.61,0.59,,0.86,0.33,0.83,0.0,39
5,68,155.0,68.0,Orthodox,6.18,0.57,4.96,0.52,,0.21,0.50,0.65,0.0,37
6,72,170.0,72.0,Orthodox,2.42,0.42,3.73,0.59,,1.20,0.38,0.75,0.4,41
